Per gli esercizi successivi, può essere utile il link alla [documentazione di OCaml del modulo List](https://v2.ocaml.org/api/List.html). Ricordiamo comunque che le funzioni higher-order viste a lezione sono
* `List.exists`
* `List.for_all`
* `List.map`
* `List.filter`
* `List.fold_left`
* `List.fold_right`

Altre funzioni del modulo `List` che potrebbero servire sono
* `List.length`
* `List.hd` (head)
* `List.tl` (tail)
* `List.rev` (reverse)
* `List.find` (trova il primo elemento che soddisfa un predicato)

**Esercizio 2.** Scrivere una funzione `zip list1 list2` che, date due liste, formi una lista con le coppie di elementi corrispondenti di `list1` e `list2`. Si deve fermare non appena una delle due liste non ha più elementi.

Per esempio, `zip [1; 2; 3; 4; 5] ['a'; 'b'; 'c']` deve calcolare `[(1, 'a'); (2, 'b'); (3, 'c')]`.

In [1]:
(* Soluzione 1: usiamo il pattern matching per destrutturare le due liste *)
let rec zip1 list1 list2 =
  match list1 with
    | [] -> []
    | x::xs -> match list2 with
      | [] -> []
      | y::ys -> (x, y)::zip1 xs ys;;

(* Soluzione 2: come per la soluzione precedente, ma uniamo i due pattern matching
 * in uno solo sulla coppia
 *)
let rec zip2 list1 list2 =
  match list1, list2 with
    | [], _ -> []
    | _, [] -> []
    | x::xs, y::ys -> (x, y)::zip2 xs ys;;

(* Soluzione 3: invece del pattern matching, usiamo un if per controllare se le liste
 * sono vuote, poi List.hd e List.tl per estrarre testa e coda delle liste
 *)
let rec zip3 list1 list2 =
  if list1 = [] || list2 = []
  then []
  else (List.hd list1, List.hd list2)::zip3 (List.tl list1) (List.tl list2);;

zip1 [1; 2; 3; 4; 5] ['a'; 'b'; 'c'];;
zip2 [1; 2; 3; 4; 5] ['a'; 'b'; 'c'];;
zip3 [1; 2; 3; 4; 5] ['a'; 'b'; 'c'];;

val zip1 : 'a list -> 'b list -> ('a * 'b) list = <fun>


val zip2 : 'a list -> 'b list -> ('a * 'b) list = <fun>


val zip3 : 'a list -> 'b list -> ('a * 'b) list = <fun>


- : (int * char) list = [(1, 'a'); (2, 'b'); (3, 'c')]


- : (int * char) list = [(1, 'a'); (2, 'b'); (3, 'c')]


- : (int * char) list = [(1, 'a'); (2, 'b'); (3, 'c')]


**Esercizio 3.** Scrivere, __possibilmente__ senza ricorsione esplicita, una funzione `enumerate list` che, data una lista, ritorni una lista di coppie che associano ad ogni elemento della lista originale il suo indice.

Per esempio, `enumerate ['a'; 'b'; 'c']` deve calcolare `[(0, 'a'); (1, 'b'); (2, 'c')]`.

In [2]:
let snd (a, b) = b;;

(* Soluzione 1: usiamo fold_left per creare la nuova lista. Questo è un pattern ricorrente,
 * e si sviluppa in questo modo. Per prima cosa, ricordiamo il tipo
 * List.fold_left :: ('a -> 'b -> 'a) -> 'a -> 'b list -> 'a
 * Questo significa che il tipo di ritorno di fold_left è quello dell'accumulatore:
 * quindi l'accumulatore deve essere una lista (visto che il nostro risultato dev'essere una
 * lista). D'altra parte, nell'accumulatore vogliamo anche tenere l'informazione su qual'è
 * l'indice dell'elemento corrente, che è un intero.
 * Per conciliare queste due necessità, usiamo come accumulatore una coppia (count, new_list)
 * in cui il primo elemento è il contatore e il secondo è la lista che stiamo costruendo. In
 * questo modo, ci basta aumentare il contatore di 1 ogni volta che esaminiamo un elemento,
 * e aggiungere alla lista che stiamo creando la nuova coppia (count, elem).
 * Alla fine, questo ci produce una coppia (lunghezza della lista, nuova lista). Visto che
 * come risultato ci interessa solo la nuova list, utiliziamo snd per estrarre solo il secondo
 * elemento della coppia.
 *)
let enumerate1 list =
  snd (List.fold_left (fun (count, new_list) elem -> (count + 1, new_list@[(count, elem)])) (0, []) list);;

enumerate1 ['a'; 'b'; 'c'];;

(* Soluzione 2: definiamo una funzione ausiliaria per crearci una lista dei numeri da 0 a
 * (n - 1), e poi usiamo zip (esercizio precedente) per zippare insieme la lista iniziale
 * e quella con gli indici corrispondenti
 *)
let lista_num n =
  let rec aux i = if i < n then i::aux (i + 1) else []
  in aux 0;;

let enumerate2 list = zip1 (lista_num (List.length list)) list;;

enumerate1 ['a'; 'b'; 'c'];;
enumerate2 ['a'; 'b'; 'c'];;

val snd : 'a * 'b -> 'b = <fun>


val enumerate1 : 'a list -> (int * 'a) list = <fun>


- : (int * char) list = [(0, 'a'); (1, 'b'); (2, 'c')]


val lista_num : int -> int list = <fun>


val enumerate2 : 'a list -> (int * 'a) list = <fun>


- : (int * char) list = [(0, 'a'); (1, 'b'); (2, 'c')]


- : (int * char) list = [(0, 'a'); (1, 'b'); (2, 'c')]


**Esercizio extra**: implementare `mapi` (già presente nella libreria `List` di OCaml). Questa funzione deve avere tipo `mapi : (int -> 'a -> 'b) -> 'a list -> 'b list`. `mapi` si comporta come `map`, con l'unica differenza che la funzione prende come argomento, oltre all'elemento della lista, anche il suo indice.

In [3]:
let mapi f list =
  List.map f (enumerate1 list);;

val mapi : (int * 'a -> 'b) -> 'a list -> 'b list = <fun>


**Esercizio 4.** Scrivere, __possibilmente__ senza ricorsione esplicita, una funzione `every n list` che prenda dalla lista `list` solo gli elementi con indice multiplo di `n`. Ricordare che gli indici partono da 0.

Per esempio, `every 3 [0; 1; 2; 3; 4; 5; 6; 7; 8; 9]` deve calcolare `[0; 3; 6; 9]`.

In [4]:
let snd (a, b) = b;;

(* Soluzione 1: usiamo fold_left come nella soluzione 1 dell'esercizio precedente.
 * L'accumulatore è una coppia (count, new_list). Se il contatore (che ci dice l'indice
 * dell'elemento corrente) è multiplo di n, aggiungiamo l'elemento alla lista,
 * altrimenti la teniamo invariata. Alla fine usiamo snd per prendere solo la nuova
 * lista e scartare il contatore.
 *)
let every1 n list =
  snd (
    List.fold_left (
      fun (count, new_list) elem -> if count mod n = 0 then (count + 1, new_list@[elem]) else (count + 1, new_list)
    ) (0, []) list
  );;

(* Soluzione 2: definiamo every n list usando funzioni higher-order.
 * Per prima cosa, usiamo enumerate per associare ad ogni elemento della lista il suo indice.
 * Poi usiamo filter per tenere solo gli elementi con indice multiplo di n.
 * Infine, usiamo map per buttare via gli indici.
 *)
let every2 n list =
  List.map snd (List.filter (fun (i, _) -> i mod n = 0) (enumerate1 list));;

every1 3 [0; 1; 2; 3; 4; 5; 6; 7; 8; 9];;
every1 3 ['a'; 'b'; 'c'; 'd'];;

val snd : 'a * 'b -> 'b = <fun>


val every1 : int -> 'a list -> 'a list = <fun>


val every2 : int -> 'a list -> 'a list = <fun>


- : int list = [0; 3; 6; 9]


- : char list = ['a'; 'd']


**Esercizio 5.** Scrivere, __possibilmente__ senza ricorsione esplicita, una funzione `no_dupl list` che ritorni `false` se la lista `list` contiene due elementi _consecutivi_ uguali, pensata come lista circolare (quindi anche il primo e l'ultimo devono essere diversi).

In [5]:
(* Soluzione 1: usiamo fold_right. Di nuovo, il pattern è lo stesso degli esercizi
 * precedenti: vogliamo calcolare il booleano che ci dice se ci sono due elementi consecutivi
 * uguali, ma non possiamo farlo analizzando un singolo elemento alla volta: ci serve anche
 * l'informazione sull'elemento successivo (perché usiamo fold_right, quindi scorriamo la
 * lista al contrario).
 * Procediamo quindi come prima: l'accumulatore è una coppia (elemento successivo, risultato
 * parziale), da cui possiamo facilmente calcolare il nuovo elemento successivo (che è elem,
 * ovvero l'elemento che stiamo processando adesso - ovvero il successore del prossimo
 * elemento che analizzeremo) e il risultato paziale (se i due elementi sono uguali false,
 * altrimenti dobbiamo semplicemente passare avanti il risultato parziale precedente, perché
 * potremmo aver già trovato una coppia di elementi consecutivi uguali prima).
 * Come sempre, concludiamo con snd per prendere solo il risultato e scartare l'informazione
 * aggiuntiva che ci serviva solo per il calcolo.
 *)
let no_dupl1 list =
  snd (
    List.fold_right (
      fun elem (succ, res) -> if elem = succ then (elem, false) else (elem, res)
    ) list (List.hd list, true)
  );;

(* Soluzione 2: usiamo funzioni higher-oder.
 * Nota importante: questa soluzione non è perfettamente funzionale e non rispetta
 * la specifica della lista ciclica. Viene inserita solamente come esempio concettuale
 * di come usare le funzioni higher-order per risolvere questo problema.
 *
 * La funzione enumerate ovvia al problema di avere l'indice trasformando ogni elemento
 * della lista in una coppia, in modo da poter applicare funzioni higher-order che operano
 * su un singolo elemento alla volta avendo però anche l'informazione non "locale" (ovvero
 * che non puoi ottenere dal solo elemento, ma che necessità anche del resto della lista)
 * della posizione.
 * In questo caso, abbiamo un problema analogo: oltre all'elemento, ci serve anche sapere
 * un'informazione non locale, ovvero l'elemento successivo. Per farlo, creiamo inizialmente
 * una nuova lista delle coppie di elementi consecutivi nella lista di partenza.
 * Per creare questa lista, usiamo la funzione zip tra list e la coda di list:

                List.tl
[1; 3; 10; 0] ----------> [3; 10; 0]

list            [1     ; 3      ; 10     ; 0]
List.tl list    [3     ; 10     ; 0      ]
                           |
                           | zip
                           v
zip             [(1, 3); (3, 10); (10, 0)]

 * Ora ci basta controllare se tutte queste coppie sono formate da elementi diversi: possiamo
 * inizialmente usare una List.map per ridurre queste coppie a dei booleani confrontando gli
 * elementi delle coppie, e poi controlliamo se tutti gli elementi della lista sono true o se
 * c'è almeno un false. (In realtà la funzione List.for_all include già una map al suo interno,
 * quindi non dobbiamo scriverla esplicitamente, ma concettualmente è quello che stiamo facendo).
 *)
let no_dupl2 list =
  List.for_all (fun (a, b) -> a != b) (zip1 list (List.tl list));;

no_dupl1 [1; 3; 10; 1];;
no_dupl1 [1; 3; 10; 0];;

val no_dupl1 : 'a list -> bool = <fun>


val no_dupl2 : 'a list -> bool = <fun>


- : bool = false


- : bool = true


**Nota finale:**
Negli esercizi precedenti, abbiamo usato le `fold` (sia `left` che `right`) con un patter abbastanza comune: vogliamo calcolare un certo _valore_ a partire dalla lista (le due nuove liste per gli es. 3 e 4, il booleano risultato nell'es. 5), ma oltre al valore accumulato fino a quel punto abbiamo bisogno anche di _informazioni aggiuntive_ (l'indice dell'elemento negli es. 3 e 4, l'elemento successivo nell'es. 5).
Per fare questo, il pattern prevede di usare come accumulatore una __coppia__ che contenga sia il valore risultato che l'informazione aggiuntiva. In questo modo, la funzione che processa un singolo elemento ha acesso sia al risultato parziale fino a quel momento (es. la lista da generare) che all'informazione aggiuntiva (es. il contatore). Si deve quindi occupare di produrre una nuova __coppia__, con il nuovo risultato parziale (la lista da generare, estesa con il nuovo valore) e l'informazione aggiuntiva aggiornata (il contatore incrementato).
Il valore calcolato dalla `fold` alla fine è quindi una coppia, di cui però ci interessa solo un elemento (la lista), mentre l'altro (il contatore) era informazione aggiuntiva necessaria solo per il calcolo: usiamo quindi la funzione di utilità `snd` (o `fst`, se ci interessa il primo elemento della coppia) per selezionare il valore che ci interessa.

**Fold, graficamente:**
segue una rappresentazione grafica di `fold_left` e `fold_right`. `z` è il valore iniziale dell'accumulatore, e `f` è la funzione che prende l'accumulatore corrente e lo "fonde" con un elemento della lista per creare il nuovo accumulatore. Fonte: [pagina su fold di HaskellWiki](https://wiki.haskell.org/Fold)

![Fold left](fold-left.png "Fold left")

![Fold right](fold-right.png "Fold right")